In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from glob import glob
import mysql.connector

import time
from bs4 import BeautifulSoup
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import json
import folium
import warnings
warnings.simplefilter(action="ignore",category=FutureWarning)
import numpy as np
import folium
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urljoin

import urllib
from urllib.request import Request
import re

In [9]:
df1 = pd.read_csv('../data/main/recruit.csv')
df2 = pd.read_csv('../data/main/recruit_data.csv')

merged = pd.concat([df1,df2])


merged.to_csv('../data/main/merged_recruit.csv', index=False)

In [10]:
df = df = pd.read_csv('../data/main/merged_recruit.csv')

education_mapping = {
    "대졸이상": 1,
    "초대졸이상": 2,
    "학력무관": 3,
    "석사이상": 4,
    "박사이상": 5,
    "고졸이상": 6
    
}

# 학력 컬럼을 매핑 사전으로 변환
df["학력"] = df["학력"].map(education_mapping)

df["학력"] = df["학력"].fillna(7)

df.to_csv('../data/main/merged_recruit_ver2.csv', index=False)

In [11]:
df = pd.read_csv('../data/main/merged_recruit_ver2.csv')

df['경력'] = df['경력'].str.replace('신입·경력1년↑', '신입', regex=False)
df['경력'] = df['경력'].str.replace('신입·경력2년↑', '신입', regex=False)
df['경력'] = df['경력'].str.replace('신입·경력3년↑', '신입', regex=False)
df['경력'] = df['경력'].str.replace('↑', '이상', regex=False)
df['경력'] = df['경력'].str.replace('신입·경력', '신입', regex=False)
df['경력'] = df['경력'].replace(r'~.*', '년이상',regex=True)
df['경력'] = df['경력'].replace('경력', '경력1년이상', regex=False)
df['경력'] = df['경력'].replace(' ', '', regex=True)

df.to_csv('../data/main/merged_recruit_ver2_1.csv',index = False)

In [12]:
career_mapping = {
    '신입': 1,
    '경력1년이상': 2,
    '경력2년이상': 3,
    '경력3년이상': 4,
    '경력4년이상': 5,
    '경력5년이상': 6,
    '경력6년이상': 7,
    '경력7년이상': 8,
    '경력8년이상': 9,
    '경력10년이상': 10,
    '경력16년이상': 11,
    '초대졸이상': 12,
    '경력무관': 13,
    None : 14
}

df = pd.read_csv('../data/main/merged_recruit_ver2_1.csv')

df['경력'] = df['경력'].map(career_mapping)

df.to_csv('../data/main/merged_recruit_ver3.csv',index = False)

In [13]:
df = pd.read_csv('../data/main/merged_recruit_ver3.csv')

company_names = [
    "클로봇", "로보티즈", "마루에이치알", "휴림로봇", "세스텍", "미르이즈", "엠앤디", "아이지", "캠틱종합기술원", 
    "원익로보틱스", "페르소나에이아이", "트위니", "휴로틱스", "두산로보틱스", "인티그리트", "럭스로보", "빅웨이브로보틱스", 
    "링크솔루션", "베어로보틱스코리아", "코가로보틱스", "인티맥스", "타스글로벌", "엔티렉스", "라스테크", "로보케어", 
    "써로마인드", "아이엠알", "스마트크리에이터", "바질컴퍼니", "도구공간", "나야넷", "이안하이텍", "로보원", "디엠에스", 
    "다운", "아인잡", "테이아", "딥마인드플랫폼", "바램시스템", "현성", "긱플러스코리아", "비아", "세성", "쉐코", "버즈니", 
    "무진기연", "스파시움로보틱스", "토트", "코리아소프트", "리릭코리아", "동원테크", "케이에이앤", "아이브이에스IVS", "에그", 
    "원진비엠티", "케이아이", "아폴로로보틱스", "디엠비에이치", "포미", "엠티에스이", "신라시스템", "지나테크", "에이티엠", 
    "이노드", "씨앤엠로보틱스", "우성기계", "소디스", "모션디바이스", "성호하이텍", "지성솔루션", "솔코", "로봇템", 
    "앤에이치씨", "애니토이", "아코스쿨", "좋은기술", "캐즈테크", "우리", "바이로봇", "로빅스", "옐로펀트로봇", 
    "블루모션테크", "지에스이", "씨엘에프하이텍", "에스엘엠", "로보스", "오토피디아", "에이버츄얼", "엣지크로스", 
    "언맨드솔루션", "로드러너", "팀로보틱스", "제이솔루션", "오션미래기술", "니어솔루션", "넷온", "디엔비소프트", "넥스콤", 
    "아이엔지글로벌", "코리아비티에스", 
    "바스토테크놀로지스", "한스콤정보통신", "엠텍정보기술", "경희정보테크", "유니드", "코리아테스팅", "레몬소프트", 
    "다울피피티", "퓨처아이스", "창의와교육", "한국엔에스디", "퓨너스", "세계로시스템", "지오소나", 
    "오픈네트웍시스템", "로보스텍", "다원테크", "아리온통신", "에로바테크놀러지", "젠아트", "태경자동화", 
    "에스에스월드", "엔티로보틱스", "엘피텍", "실루엣미디어그룹아시아", "에이더블테크놀러지", "와이즈박스", 
    "메디피아이앤씨", "팩스코", "아주실업", "엑스로보", "삼익티엔에이치", "두리씨앤에스", "올더테크", 
    "스마트시스템즈", "로시", "와이지솔루션", "네오피아", "알엔티", "에임폴", "㈜디에스티로봇", "로보로텍", 
    "한국지기", "엔아이스퀘어", "로아스", "아르고스다인", "그린로보텍", "에스브이", "한컴어썸텍", "키네틱스", 
    "테이슨", "아원기술", "모디", "조은에듀테크", "스토브리코리아", "헬퍼로보틱스", "스튜디오랩", 
    "비욘드허니컴", "시아", "아비만로보랩 주식회사", "로보다인시스템", "크레듀", "마젠타로보틱스", 
    "아카에이아이", "하이제라네트웍스", "쿳션", "알오지스틱스", "킴스트", "제이비소프트", "클레빌", 
    "로보웍스", "디웹스", "지엔디비즈", "아디아랩", "에이아이에스", "레디츠", "에스와이이노테크", "페이텀", 
    "제이지파워넷", "따따스튜디오", "아이오테드", "써니팩토리", "나란", "디자인리퍼블릭", "넷솔루션", 
    "에프엠", "유에이로보틱스", "윔", "라파비", "오토기어드", "에이아이솔텍", "로봇앤모어", "신라정보기술", 
    "케이피씨전자", "로보그램인공지능로봇연구소", "로보트론", "주원테크놀러지", "라이트테크", "플롯팩토리", 
    "엣눅하다",
    "아이알로봇", "오랩스", "쓰리에이솔루션즈", "리맵", "에스피엘티", "마린로보틱스", "아프스", "로보그램", 
    "드론메이커스", "네오아크로보틱스", "삼영기술", "티포엘", "절벽의오른쪽", "플로리젠로보틱스", 
    "주신에코솔루션", "주재영에이아이컴퍼니", "로보이", "로비텍", "에이티엠솔루션", "내비오닉스코리아", 
    "로보틱박스", "카이렌", "글피", "아덴트로봇", "우술", "지능로봇스튜디오", "드림가이드컴퍼니", 
    "한아시스템", "하이젠파워", "스마트로봇연구소", "이지텍", "무브투", "디로펫", "에이치오엔터테인먼트", 
    "프렉시온", "에프티 드론", "쉐프로봇테크", "씨앤플러스", "에스에이치로보틱스", "아이피엠로보틱스", 
    "에스아이웨어", "워킨위드", "한국영상솔루션", "도안앤텍", "주비스", "에스에스알", "마인드로", "두이엔지", 
    "새비로보틱스", "디앤씨", "로닉", "다비다", "에프티에스", "세종뉴텍", "에스제이워터테크", "드림로보", 
    "아이티융합교육개발원", "넥스젠", "유니아이", "바이클립", "산타마리아(SantaMaria)", "이스타이엔지", 
    "이드엠", "가온아이티", "일렉테크코리아", "르호봇알브이엠", "시드크래프트", "엘디에이치", "엠엔지", 
    "어반시드디자인협동조합", "니즈로봇", "피앤엘글로벌", "로보플래닛", "링크투", "베이스스톤", "영산수도건설", 
    "알이랩", "신진기술", "쓰리코엔지니어링", "엔에스로보텍", "세이프티랩", "테파로보틱스", "에이콘", 
    "이팩토리", "리겔", "대덕로보트", "에스티앤아이", "스마트시스템", "모스트아이텍", "포어사이트", 
    "더린킨", "디와이오토메이션", "엘로이텍", "지노테크", "케이브", "짓필라엠", "비앤디", "도아로보틱스", 
    "디티패스", "지능디자인",
    "효원", "에이원랩", "지플레인", "어반솔루션", "themasters", "로그틱스", "이로보텍", "거산테크", "스카이로보", 
    "씨에스랩", "에스엔소프트", "모두의VR", "모멘텀스카이", "아이알씨", "에듀로봇", "라임랩", "두성씨엔에이", 
    "미담하이웍스", "피두스젠", "라온네트웍스", "러닝퓨처", "제이알씨", "스터디랜드", "다빈치창의융합연구소", 
    "어크로스페이스에듀", "에이치엑스", "도우마리", "디봄", "오렌지", "미래항공기술", "오토르", "아이오티테크", 
    "한국농업기술지주", "동아F.A", "스티그마", "예솜", "핸드메이드", "에이아이그릭", "에스엠제이", "나우닷", "보라", 
    "꿈나무이야기", "윈드로로보틱스", "홍익팜", "누빈다Nuvinda", "유캔맥스", "에이엔케이이노베이션", "월드지엔유", 
    "스팀메이커", "인공지능과사람들", "티엑스솔루션", "이지정보통신", "에스앤제이", "케이제이피기술", "에스브이티", 
    "월넛", "스카이드림", "제로클래스랩", "케이디엔알", "에이치엔알", "노아랩", "더컬쳐", "다올메이커스", 
    "애그리펑션", "한다코퍼레이션", "가든테크", "시냅로봇", "제이엘유", "리크라", "에코트로닉스", "로보케어", 
    "라이온로보틱스", "RGA Inc.", "아이아이컴바인드", "대동모빌리티", "로보터스", "영현로보틱스", "지아이티", 
    "세종교육", "하이비젼시스템", "경기도일자리재단", "한화로보틱스", "로보티즈", "플라잎", "메디스비", "엔케이알", 
    "토르드라이브", "유진로봇", "모빌테크", "엔티렉스", "리브스메드", "애드인에듀", "폴라리스쓰리디", "태경전자", 
    "고레로보틱스", "와이즈교육", "레인보우로보틱스", "에이블랩스", "내비간티스코리아", "씨메스",
    "나우로보틱스", "한국폴리텍대학분당융합기술교육원", "인아텍앤코포", "에스티이씨", "하이텍알씨디코리아", 
    "칼만텍", "티라로보틱스", "로아스", "가온브로드밴드", "큐라코", "와따에이아이", "이엘피", 
    "슈어소프트테크", "현대오토에버", "덴티움", "시스윈로보틱스", "코아텍", "여의시스템", "두림야스카와", 
    "앤씨앤", "코웨이", "엘로이랩", "바이트사이즈", "빅웨이브로보틱스", "나비프라", "알티올", 
    "시스윈모빌리티", "씨드로닉스", "에스엠소프트", "대한상공회의소 서울기술교육센터", "테오시스", 
    "로보에테크놀로지", "엑스와이지로보틱스코리아 유한회사", "에이아이엠", "씨트론", "하이버스이앤씨", 
    "에스티에스", "퀀텀에어로", "알지티", "피지오닉스", "에이로봇", "쉐코", "크래블", "퓨전테크놀로지", 
    "아이에스피", "애니모션텍", "아테코", "럭스피엠", "에이블랩스(ABLE Labs Inc.)", "에니아이", "델타엑스", 
    "브릴스", "넥스틴", "지아이웍스", "하이퍼오토메이션", "포디아이비젼", "지뉴소프트", "프리콘", "아이로", 
    "와이즈에이아이", "파익스", "영풍전자", "씨메트", "한국브룩스오토메이션", "큐버", "오키", "디비엔텍", 
    "바리코리아", "너츠앤씨웨이", "캐리마", "메타빈스", "크레템", "뷰케이코리아", "니어스랩", "지비넷", 
    "싸이맥스", "애드인소프트", "비밀리", "보스반도체", "아이브", "벤디트", "피앤에스미캐닉스", "메뉴톡", 
    "인피닉", "모벤시스", "에이딘로보틱스", "3D센서 기반의 머신비전 연구 전문기업", "대기업", 
    "대동에크테크", "잡뉴스솔로몬서치", "에어스", "베스텔라랩", "옵트에이아이", "에어스", "베스텔라랩", 
    "옵트에이아이", "에어스", "베스텔라랩", "옵트에이아이", "유토시스",
    "스파크엑스", "에이피텍", "에이텍오토", "디아이티", "잇츠센서", "로보터블", "라이브워크", 
    "로보아이", "에스엠아이", "비전메카", "제이디", "엑스오비스", "에프에스솔루션", "이모션텍", 
    "유엔디", "뮤텍코리아", "코트라스", "알씨테크", "아이티엔제이", "에이트테크", "경신", 
    "레이언스", "에브리봇", "휴비스", "테크윙", "엔바이어스", "밸류플러스테크놀러지", "다스코리아", 
    "로봇앤드디자인", "아이오크롭스", "와이파워원", "프레스토솔루션", "모핑아이", "미스터마인드", 
    "테스콤", "씨에이", "씨유박스", "두원전자", "울산대학교산학협력단", "화인코왁", "크레셈", 
    "오토엘", "에이치알캡", "아이로바", "쎄믹스", "드림에이스", "시에라베이스", "울랄라랩", 
    "위고로보틱스", "엘라인"

]

df["회사명"] = df["회사명"].str.strip()

company_mapping = {name: index + 1 for index, name in enumerate(company_names)}


df['회사명'] = df['회사명'].map(company_mapping)

df.to_csv('../data/main/merged_recruit_ver4.csv',index = False)

In [ ]:
conn = mysql.connector.connect(
    host = "database-1.c9ks828mszs7.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "asd20187304",
    database = "roschool"
)
df = pd.read_csv('../data/merged_recruit_final.csv')
df = df.where(pd.notnull(df), None)
cursor = conn.cursor(buffered=True)
sql = """insert into recruit_info (comp_name_id, career_id,education_id) values (%s, %s, %s)"""

for row in df.itertuples(index=False, name=None):
    cursor.execute(sql, row)
    conn.commit()



In [ ]:
conn.close()

In [14]:
df = pd.read_csv('../data/main/essential_tech_info.csv')
df
df["필수사항"] = df["필수사항"].str.split(",")
df = df.explode("필수사항")
df['필수사항'] = df['필수사항'].str.strip()

df.to_csv('../data/main/essential_tech_info_ver1.csv', index=False)

In [15]:
df = pd.read_csv('../data/main/prefer_tech_info.csv')
# df
df['우대사항'].fillna(117, inplace=True)

df["우대사항"] = df["우대사항"].str.split(",")
df = df.explode("우대사항")
df['우대사항'] = df['우대사항'].str.strip()

df.to_csv('../data/main/prefer_tech_info_ver1.csv', index=False)

In [16]:
df = pd.read_csv('../data/main/essential_tech_info_ver1.csv')

df['필수사항'] = df['필수사항'].str.replace('임베디드', 'Embedded', regex=False).str.replace('EMbedded', 'Embedded', regex=False).str.replace('Embeded', 'Embedded', regex=False)
df['필수사항'] = df['필수사항'].str.replace('Java script', 'Java', regex=False).replace('javascript', 'Java', regex=False)
df['필수사항'] = df['필수사항'].str.replace('Jiar', 'Jira', regex=False)
df['필수사항'] = df['필수사항'].str.replace('NET framework', '.NET', regex=False).str.replace('Net Framework', '.NET', regex=False)
df['필수사항'] = df['필수사항'].str.replace(' NO SQL', 'NoSQL', regex=False).str.replace('No-SQL', 'NoSQL', regex=False)
df['필수사항'] = df['필수사항'].str.replace('Pytorch', 'PyTorch', regex=False).str.replace('pyTorch', 'PyTorch', regex=False)
df['필수사항'] = df['필수사항'].str.replace('Tensorflow', 'TensorFlow', regex=False)
df['필수사항'] = df['필수사항'].str.replace('ROS1', 'ROS', regex=False).str.replace('ROS2', 'ROS', regex=False)
df['필수사항'] = df['필수사항'].str.replace('Ubuntu', 'Linux', regex=False)
df['필수사항'] = df['필수사항'].str.replace('Matlab', 'MATLAB', regex=False)


df.to_csv('../data/main/essential_tech_info_ver2.csv',index = False)

In [17]:
df = pd.read_csv('../data/main/prefer_tech_info_ver1.csv')

df['우대사항'] = df['우대사항'].str.replace('임베디드', 'Embedded', regex=False).str.replace('EMbedded', 'Embedded', regex=False).str.replace('Embeded', 'Embedded', regex=False)
df['우대사항'] = df['우대사항'].str.replace('Java script', 'Java', regex=False).replace('javascript', 'java', regex=False)
df['우대사항'] = df['우대사항'].str.replace('Jiar', 'Jira', regex=False)
df['우대사항'] = df['우대사항'].str.replace('NET framework', '.NET', regex=False).str.replace('Net Framework', '.NET', regex=False)
df['우대사항'] = df['우대사항'].str.replace('NO SQL', 'NoSQL', regex=False).str.replace('No-SQL', 'NoSQL', regex=False)
df['우대사항'] = df['우대사항'].str.replace('Pytorch', 'PyTorch', regex=False).str.replace('pyTorch', 'PyTorch', regex=False)
df['우대사항'] = df['우대사항'].str.replace('Tensorflow', 'TensorFlow', regex=False)
df['우대사항'] = df['우대사항'].str.replace('ROS1', 'ROS', regex=False).str.replace('ROS2', 'ROS', regex=False)
df['우대사항'] = df['우대사항'].str.replace('Ubuntu', 'Linux', regex=False)
df['우대사항'] = df['우대사항'].str.replace('Matlab', 'MATLAB', regex=False)


df.to_csv('../data/main/prefer_tech_info_ver2.csv',index = False)

In [20]:
tech_list = [
    ".NET", "AGV", "AMR", "API", "ARM", "ASP.NET", "AVR", "AWS", "Agile", "Android",
    "Arduino", "Asana", "Azure", "BLDC", "BLE", "C", "C#", "C++", "CAN", "CANoe", 
    "CARLA", "CIM", "CSS", "CUDA", "Cloud", "Conda", "DBMS", "DL", "Delphi", "DevOps",
    "DoD", "Docker", "Embedded", "EtherCAT", "Flutter", "GCS", "GEM", "GLFW", "GTK",
    "GUI", "Gazebo", "Git", "HMI", "HTML", "Halcon", "IPC", "ISP", "Java", "Javascript",
    "Jenkins", "Jira", "Keras", "Kotlin", "Linux", "MATLAB", "MCU", "MEM", "MES", "MFC",
    "MIL", "MXNet", "MySQL", "Next.js", "NginX", "NoSQL", "Node.js", "OMS", "OpenCV", 
    "OpenGL", "Oracle", "PIC", "PostgreSQL", "PowerBI", "PyTorch", "Python", "QT", "R",
    "R&D", "RDBMS", "REST API", "RESTful", "RESTful API", "ROS", "RTOS", "RaspberryPi",
    "React", "Redis", "Redmine", "Rust", "SDK", "SDL", "SECS", "SLAM", "SPI", "SQA", 
    "SQL", "SVM", "Simulator", "Slack", "SoC", "Spring", "SpringBoot", "TCP/IP", 
    "Tableau", "TensorFlow", "UART", "UCI", "Unix", "VTK", "WCF", "WPF", "WRF", 
    "Webots", "doxyzen", "딥러닝", "정보없음"
]
df = pd.read_csv('../data/main/essential_tech_info_ver2.csv')

for idx, each in enumerate(df["필수사항"]):
    if each in tech_list:
        df["필수사항"][idx] = tech_list.index(df["필수사항"][idx]) + 1

df.to_csv('../data/main/essential_tech_info_ver3.csv', index=False)

/tmp/ipykernel_16225/2671097268.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["필수사항"][idx] = tech_list.index(df["필수사항"][idx]) + 1
/tmp/ipykernel_16225/2671097268.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["필수사항"][idx] = tech_list.index(df["필수사항"][idx]) + 1
/tmp/ipykernel_16225/2671097268.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["필수사항"][idx] = tech_list.index(df["필수사항"][idx]) + 1
/tmp/ipykernel_1622

In [22]:
df = pd.read_csv('../data/main/prefer_tech_info_ver2.csv')
for idx, each in enumerate(df["우대사항"]):
    if each in tech_list:
        df["우대사항"][idx] = tech_list.index(df["우대사항"][idx]) + 1

df.to_csv('../data/main/prefer_tech_info_ver3.csv', index=False)

/tmp/ipykernel_16225/2459062404.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["우대사항"][idx] = tech_list.index(df["우대사항"][idx]) + 1
/tmp/ipykernel_16225/2459062404.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["우대사항"][idx] = tech_list.index(df["우대사항"][idx]) + 1
/tmp/ipykernel_16225/2459062404.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["우대사항"][idx] = tech_list.index(df["우대사항"][idx]) + 1
/tmp/ipykernel_16225/2

In [ ]:
conn = mysql.connector.connect(
host = "database-1.c9ks828mszs7.ap-northeast-2.rds.amazonaws.com",
port = 3306,
user = "root",
password = "asd20187304",
database = "roschool"
)

In [ ]:
df = pd.read_csv('../data/prefer_tech_info_ver3.csv')

cursor = conn.cursor(buffered=True)
sqlr = """insert into prefer_tech (recruit_id, tech_id) values (%s, %s)"""

for i, row in df.iterrows():
    cursor.execute(sqlr, (int(row['채용정보']),int(row['우대사항'])))
    conn.commit()

In [ ]:
df = pd.read_csv('../data/essential_tech_info_ver3.csv')

cursor = conn.cursor(buffered=True)
sqlr = """insert into essential_tech (recruit_id, tech_id) values (%s, %s)"""

for i, row in df.iterrows():
    cursor.execute(sqlr, (int(row['채용정보']),int(row['필수사항'])))
    conn.commit()


In [ ]:
conn.close()